# Эффект Поккельса

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import pandas as pd
import dataframe_image as dfi
from scipy.stats import linregress